In [1]:
import os
os.chdir(os.path.join(os.getcwd(), '..'))
os.getcwd()

'/home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA'

In [2]:
from src.utils import get_dataset
from src.utils import select_data
import scipy.io as sio
import numpy as np

from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

/home/renjie/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
DATA_PATH = os.path.join(os.getcwd(), 'data')
depth_path, inertial_path, skeleton_path = get_dataset(DATA_PATH)

In [4]:
activities = list(range(1,28))

In [5]:
def select_subject(d_path, subject):
    select_statement = '_s{}_'.format(subject)
    subjects = []
    for i in d_path:
        if select_statement in i:
            subjects.append(i)
    return subjects

In [6]:
def get_action_number(single_path):
    return int(single_path.split('/')[-1].split('_')[0][1:])
def get_subject_number(single_path):
    return int(single_path.split('/')[-1].split('_')[1][1:])
def get_trial_number(single_path):
    return int(single_path.split('/')[-1].split('_')[2][1:])


In [7]:
def pad_len_inertial(inertial_d):
    inertial_d = np.swapaxes(inertial_d, 0,1)
    inertial_d = sequence.pad_sequences(inertial_d, maxlen=326)
    inertial_d = np.swapaxes(inertial_d, 0,1)
    return inertial_d

In [8]:
# train on subject 1,2,3,5,6,7
# test on subject 4,8
X_train = []
Y_train = []
X_test = []
Y_test = []

for path in inertial_path:
    if get_subject_number(path) in [1,2,3,5,6,7]:
        X_train.append(path)
        Y_train.append(get_action_number(path))
    else:
        X_test.append(path)
        Y_test.append(get_action_number(path))

X_train = [pad_len_inertial(sio.loadmat(x)['d_iner']) for x in X_train]
X_test = [pad_len_inertial(sio.loadmat(x)['d_iner']) for x in X_test]

X_train = np.array(X_train)
X_test = np.array(X_test)

Y_train = to_categorical(np.array(Y_train) - 1)
Y_test = to_categorical(np.array(Y_test) - 1)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((646, 326, 6), (646, 27), (215, 326, 6), (215, 27))

# Model

In [9]:
np.random.seed(7)
model = Sequential()
model.add(LSTM(200, input_shape=(326, 6)))
model.add(Dense(len(activities), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               165600    
_________________________________________________________________
dense_1 (Dense)              (None, 27)                5427      
Total params: 171,027
Trainable params: 171,027
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train, Y_train, epochs=100, batch_size=1)

Epoch 1/100
646/646 [==============================] - 125s 193ms/step - loss: 2.0956 - acc: 0.3483
Epoch 2/100
646/646 [==============================] - 136s 210ms/step - loss: 1.9700 - acc: 0.4149
Epoch 3/100
646/646 [==============================] - 132s 204ms/step - loss: 1.9108 - acc: 0.4102
Epoch 4/100
646/646 [==============================] - 115s 178ms/step - loss: 1.8930 - acc: 0.4226
Epoch 5/100
646/646 [==============================] - 118s 183ms/step - loss: 1.8134 - acc: 0.4613
Epoch 6/100
646/646 [==============================] - 117s 181ms/step - loss: 2.2001 - acc: 0.3808
Epoch 7/100
646/646 [==============================] - 141s 218ms/step - loss: 2.6042 - acc: 0.2198
Epoch 8/100
646/646 [==============================] - 143s 222ms/step - loss: 2.2910 - acc: 0.3344
Epoch 9/100
646/646 [==============================] - 143s 222ms/step - loss: 2.1557 - acc: 0.3514
Epoch 10/100
646/646 [==============================] - 143s 222ms/step - loss: 2.0085 - acc: 0.4133

In [11]:
X_train[0].shape

(326, 6)